In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

load_dotenv(dotenv_path='../scratch/dev.env')

model = OpenAI()

# Define your desired data structure.
class Utterances(BaseModel):
    utterances: list[str] = Field(description="The array of generated utterances.", examples=["Hello", "Hi", "Hey"])
    
parser = JsonOutputParser(pydantic_object=Utterances)

sys_prompt = """You are a helpful assistant that generates high-quality training data for use with voice and chat bots."""

base_prompt = PromptTemplate(
    template="{sys_prompt}.  Please generate {quantity} semantically diverse ways {base_phrase}.\n{format_instructions}\n",
    input_variables=["sys_prompt", "quantity", "base_phrase"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

base_chain = base_prompt | model | parser

base_chain_result = base_chain.invoke({
    "sys_prompt": sys_prompt,
    "quantity": 10,
    "base_phrase": "someone might express appreciation to a bot",
})

print(base_chain_result)

transform_prompt = PromptTemplate(
    template="{sys_prompt}.  Here is a list of utterances:\n\n{utterances}\n\nPlease repeat these examples, but {transform_phrase}.\n{format_instructions}\n",
    input_variables=["sys_prompt", "utterances", "transform_phrase"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

transform_chain = transform_prompt | model | parser

transform_chain_result = transform_chain.invoke({
    "sys_prompt": sys_prompt,
    "utterances": base_chain_result['utterances'],
    "transform_phrase": "sound more casual and use half the words",
})

print(transform_chain_result)


{'utterances': ['Thank you for your help!', 'I appreciate your assistance.', 'You are doing a great job!', 'Thanks for being so helpful.', 'I am grateful for your support.', 'You are amazing, bot!', 'I am so thankful for your service.', 'Your assistance is greatly appreciated.', 'Thank you for always being there.', 'I am so glad I have you, bot!']}
{'utterances': ['Thanks for your help!', 'Appreciate your help!', 'Great job!', 'Thanks for being helpful.', 'Grateful for your support.', "You're amazing, bot!", 'Thankful for your service.', 'Your help is greatly appreciated.', 'Thank you for always being there.', 'Glad to have you, bot!']}
